In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import regexp_extract
from functools import partial

In [2]:

spark = (
    SparkSession.builder.appName("ToParquetLeagueOfData")
    .master("spark://spark-master:7077")
    .config(
        "spark.jars.packages",
        "com.amazonaws:aws-java-sdk:1.11.563,com.amazonaws:aws-java-sdk-bundle:1.11.874,org.apache.hadoop:hadoop-aws:3.3.2,io.delta:delta-core_2.12:2.1.0"
    )
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.hadoop.fs.s3a.multipart.size", "838860800")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    )
    .config("spark.hadoop.fs.s3a.path.style.access", True)
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.amazonaws#aws-java-sdk added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ea8fc165-fa34-410a-a58e-2f50fa0a42a0;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk;1.11.563 in central
	found com.amazonaws#aws-java-sdk-ioteventsdata;1.11.563 in central
	found com.amazonaws#aws-java-sdk-core;1.11.563 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.5 in central
	found org.apache.httpcomponents#httpcore;4.4.9 in central
	found commons-codec#commons-codec;1.10 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.6.7.2 in central
	found com.fasterxml.jackson.

23/05/03 22:46:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
#spark.stop()

In [3]:
spark.sparkContext.setLogLevel("ERROR")
sc = spark
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.access.key", "minio-root-user")
hadoop_conf.set("fs.s3a.secret.key", "minio-root-password")
hadoop_conf.set("fs.s3a.path.style.access", "True")


In [4]:
matchs_df = spark.read.json("s3a://league-of-data-bronze/matchs/detail")

In [5]:
schema = matchs_df.schema

23/05/03 22:56:16 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
23/05/03 22:56:16 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:218)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:923)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [25]:
# df = spark.read.format('json').schema(schema).load("s3a://league-of-data-bronze/matchs/detail")
# df = df.select(col("info.*"), col("match"), col("extracted_at"))
# df.write.mode("overwrite").partitionBy("match", "extracted_at").format(
#     "delta"
# ).save("s3a://league-of-data-ingestion-silver-stream/stream/match/detail/")

In [26]:
def _upsert(microbatch, batch_id):
    delta_table.alias("match_details").merge(
    microbatch.alias("new_match_details"),
        "match_details.match = new_match_details.match") \
      .whenNotMatchedInsertAll() \
      .execute()

In [27]:
from delta import DeltaTable

delta_table = DeltaTable.forPath(spark, path="s3a://league-of-data-ingestion-silver-stream/stream/match/detail/")

In [28]:
df = spark.readStream.format('json').schema(schema).load("s3a://league-of-data-bronze/matchs/detail")
df = df.select(col("info.*"), col("match"), col("extracted_at"))
df.writeStream.format("delta").outputMode("update").foreachBatch(_upsWert).option("checkpointLocation", "s3a://league-of-data-ingestion-silver-stream/spark-stream-checkpoint") \
.start().awaitTermination()

23/02/26 01:03:36 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
23/02/26 01:03:36 ERROR MicroBatchExecution: Query [id = e2c5e53c-bdb3-494f-93a7-4ba208c1ff65, runId = 015f8245-0c7c-48b9-8c75-96c29f1356d9] terminated with error
org.apache.spark.SparkException: Job aborted due to stage failure: Master removed our application: KILLED
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.ap

StreamingQueryException: Query [id = e2c5e53c-bdb3-494f-93a7-4ba208c1ff65, runId = 015f8245-0c7c-48b9-8c75-96c29f1356d9] terminated with exception: Job aborted due to stage failure: Master removed our application: KILLED

In [ ]:
from pyspark.sql.functions import *
match_info_df = matchs_df.select(col("info.*"), col("match"), col("extracted_at"))

In [33]:
delta_details = spark.read.json("s3a://league-of-data-bronze/summoners/details/")

StructType([StructField('matches', ArrayType(StringType(), True), True), StructField('summoner_data', StructType([StructField('freshBlood', BooleanType(), True), StructField('hotStreak', BooleanType(), True), StructField('inactive', BooleanType(), True), StructField('leaguePoints', LongType(), True), StructField('losses', LongType(), True), StructField('rank', StringType(), True), StructField('summonerId', StringType(), True), StructField('summonerName', StringType(), True), StructField('veteran', BooleanType(), True), StructField('wins', LongType(), True)]), True), StructField('summoner_detail', StructType([StructField('accountId', StringType(), True), StructField('id', StringType(), True), StructField('name', StringType(), True), StructField('profileIconId', LongType(), True), StructField('puuid', StringType(), True), StructField('revisionDate', LongType(), True), StructField('summonerLevel', LongType(), True)]), True), StructField('summoner', StringType(), True), StructField('extrac

In [49]:
delta_details.select(col("matches")).show()

+--------------------+
|             matches|
+--------------------+
|[BR1_2656013243, ...|
|[BR1_2655785627, ...|
|[BR1_2637963534, ...|
|[BR1_2631729244, ...|
|[BR1_2652364095, ...|
|[BR1_2654184620, ...|
|[BR1_2655972514, ...|
|[BR1_2649383583, ...|
|[BR1_2655864488, ...|
|[BR1_2628440952, ...|
|[BR1_2656185048, ...|
|[BR1_2627426671, ...|
|[BR1_2655438148, ...|
|[BR1_2630238819, ...|
|[BR1_2655864488, ...|
|[BR1_2654632443, ...|
|[BR1_2656399916, ...|
|[BR1_2638463498, ...|
|[BR1_2655759196, ...|
|[BR1_2622442677, ...|
+--------------------+
only showing top 20 rows



In [53]:
summoner_matchs = delta_details.withColumn("matches", explode(delta_details.matches))
summoner_matchs.show()

+--------------+--------------------+--------------------+--------------------+------------+
|       matches|       summoner_data|     summoner_detail|            summoner|extracted_at|
+--------------+--------------------+--------------------+--------------------+------------+
|BR1_2656013243|{true, false, fal...|{Tpotr1ogQeiGHJVF...|2g5yWN_QkvNKXIQP2...|  2023-01-04|
|BR1_2655931534|{true, false, fal...|{Tpotr1ogQeiGHJVF...|2g5yWN_QkvNKXIQP2...|  2023-01-04|
|BR1_2655913647|{true, false, fal...|{Tpotr1ogQeiGHJVF...|2g5yWN_QkvNKXIQP2...|  2023-01-04|
|BR1_2655899148|{true, false, fal...|{Tpotr1ogQeiGHJVF...|2g5yWN_QkvNKXIQP2...|  2023-01-04|
|BR1_2655577787|{true, false, fal...|{Tpotr1ogQeiGHJVF...|2g5yWN_QkvNKXIQP2...|  2023-01-04|
|BR1_2655487847|{true, false, fal...|{Tpotr1ogQeiGHJVF...|2g5yWN_QkvNKXIQP2...|  2023-01-04|
|BR1_2655466591|{true, false, fal...|{Tpotr1ogQeiGHJVF...|2g5yWN_QkvNKXIQP2...|  2023-01-04|
|BR1_2655443791|{true, false, fal...|{Tpotr1ogQeiGHJVF...|2g5yWN_QkvNK

In [55]:
summoner_matchs_row_df = summoner_matchs.select(col("matches"), col("summoner_detail.puuid"), col("extracted_at"))

In [56]:
summoner_matchs_row_df.write.mode("overwrite").partitionBy("puuid", "extracted_at").format("delta").save("s3a://league-of-data-bronze/summoner/matches/")
